In [1]:
!pip install sklearn

  Using cached sklearn-0.0.post1-py3-none-any.whl


In [2]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors

In [3]:
qryMeta = """
Select *
from delorean_dataset.metadata_sitios
"""
dfMeta = pd.read_gbq(qryMeta, dialect='standard')

In [5]:
# Filtra columnas
keep_cols = ['gmap_id','restaurant_0_hotel_1','avg_rating','num_of_reviews','State','city','category','avg_compound','avg_size']

In [6]:
dfMeta_filtrado = dfMeta[keep_cols]

In [7]:
# Create pipeline para variables numericas
numeric_pipe = Pipeline([
    ('scaler', StandardScaler())
    ])

In [8]:
# Crea pipeline para procesar  variables categoricas
categorical_pipe = Pipeline([
    ('encoder', OneHotEncoder(drop = 'first'))
    ])

In [9]:
# Transforma las columnas
col_transf = ColumnTransformer([
    ('numeric', numeric_pipe, dfMeta_filtrado._get_numeric_data().columns.tolist()),
    ('categoric', categorical_pipe, dfMeta_filtrado.select_dtypes('object').columns.tolist()) 
    ])

In [15]:
dfMeta_filtrado.fillna(0, inplace=True)

In [16]:
col_transf_fit = col_transf.fit(dfMeta_filtrado)
dfMeta_filtrado_transf = col_transf_fit.transform(dfMeta_filtrado)
dfMeta_filtrado_transf

<167150x202597 sparse matrix of type '<class 'numpy.float64'>'
	with 1140900 stored elements in Compressed Sparse Row format>

In [ ]:
# Clasificación o regresión basada en vecinos cercanos

In [21]:
n_neighbors=5

In [24]:
# Algoritmo NearestNeighbors de Scikit-learn
#Este parámetro especifica el número de vecinos más cercanos que se utilizarán para encontrar los puntos más cercanos a un punto de consulta
nneighbors = NearestNeighbors(n_neighbors = n_neighbors, metric = 'cosine').fit(dfMeta_filtrado_transf)

In [25]:
dif, ind = nneighbors.kneighbors(dfMeta_filtrado_transf[1])

In [26]:
print("Lugares que ha gustado")
print("="*80)
dfMeta.loc[ind[0][0], :]
print("Lugares recomendados")
print("="*80)
dfMeta.loc[ind[0][1:], :]

Lugares que ha gustado
Lugares recomendados


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,...,MISC,relative_results,url,State,city,restaurant_0_hotel_1,characteristics,categoria,avg_compound,avg_size
1,village parkway 03663,"village parkway 03663, 03663 buchanan st space...",0x80dcb77a2a495555:0x554654212d520928,no especifica,33.888975,-117.496585,'bar',4.5,2,no especifica,...,"{'service options': ['dine-in'], 'accessibilit...","'0x80dcb771cac37b5b:0xba8d4ef9cd2f75fd', '0x80...",https://www.google.com/maps/place//data=!4m2!3...,CA,riverside,0,'',None,NaN,NaN
642,vino veritas llc,"vino veritas llc, 285 alessandro blvd ste 7f, ...",0x80dca57b6f411887:0xa98b7601746a7de7,no especifica,33.915375,-117.328823,'bar',4.3,3,no especifica,...,"{'atmosphere': ['casual', 'cozy', 'upscale'], ...","'0x80dcb1f00be31b9b:0x12a2fa6edbf5d9a9', '0x80...",https://www.google.com/maps/place//data=!4m2!3...,CA,riverside,0,'',None,NaN,NaN
657,racers pub,"racers pub, 3847 pierce st, riverside, ca 92503",0x80dcb760409df5a7:0xbba7f77fa9c2fb4e,no especifica,33.896786,-117.490542,'bar',4.2,58,no especifica,...,"{'service options': ['delivery'], 'health & sa...","'0x80dcb771cac37b5b:0xba8d4ef9cd2f75fd', '0x80...",https://www.google.com/maps/place//data=!4m2!3...,CA,riverside,0,'',None,NaN,NaN
72749,catrina tequila bar,"catrina tequila bar, 34 east ave, riverside, i...",0x880e357eb118c15d:0xa6747c82f74d1314,no especifica,41.827925,-87.820552,'bar',4.9,19,no especifica,...,"{'service options': ['curbside pickup', 'takeo...",nan,https://www.google.com/maps/place//data=!4m2!3...,IL,riverside,0,'',None,NaN,NaN
